In [1]:
import os
import logging
from datetime import datetime

In [2]:


import boto3
import sagemaker
from sagemaker.session import TrainingInput
from sagemaker import image_uris
from sagemaker import hyperparameters



sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\hemch\AppData\Local\sagemaker\sagemaker\config.yaml


In [3]:
boto3.set_stream_logger(name="botocore.credentials", level=logging.WARNING)

In [4]:
region = sagemaker.Session().boto_region_name
print(region)

us-east-1


In [5]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

import json
import logging
import sys
from pathlib import Path

import ipytest

#CODE_FOLDER = Path("code")
#sys.path.extend([f"./{CODE_FOLDER}"])

#DATA_FILEPATH = "../data/penguins.csv"

ipytest.autoconfig(raise_on_error=True)

# By default, The SageMaker SDK logs events related to the default
# configuration using the INFO level. To prevent these from spoiling
# the output of this notebook cells, we can change the logging
# level to ERROR instead.
logging.getLogger("sagemaker.config").setLevel(logging.ERROR)

In [7]:
import os

bucket = os.environ["BUCKET"]+'/sagemaker-course'
role = os.environ["ROLE"]

COMET_API_KEY = os.environ.get("COMET_API_KEY", None)
COMET_PROJECT_NAME = os.environ.get("COMET_PROJECT_NAME", None)

In [8]:
bucket

'beercafe-ml-bucket/sagemaker-course'

In [9]:
prefix = "iris"

In [10]:
!aws s3 ls {bucket}/{prefix}/

                           PRE batch_transform/
                           PRE data/


In [11]:
!aws s3 ls {bucket}/{prefix}/data/ --recursive

2024-04-16 21:40:09        950 sagemaker-course/iris/data/iris_test.csv
2024-04-16 21:40:00       1900 sagemaker-course/iris/data/iris_train.csv


In [12]:
train_file = "data/iris_train.csv"
valid_file = "data/iris_test.csv"

train_file_uri = "s3://{}/{}/{}".format(bucket, prefix, train_file)
valid_file_uri = "s3://{}/{}/{}".format(bucket, prefix, valid_file)
print("train file uri:", train_file_uri)
print("valid file uri:", valid_file_uri)

train file uri: s3://beercafe-ml-bucket/sagemaker-course/iris/data/iris_train.csv
valid file uri: s3://beercafe-ml-bucket/sagemaker-course/iris/data/iris_test.csv


In [14]:
train_ip = TrainingInput(train_file_uri, content_type="csv")
valid_ip = TrainingInput(valid_file_uri, content_type="csv")
print(train_ip)
print(valid_ip)

In [15]:
model_op = "s3://{}/{}/{}".format(bucket, prefix, "model")
print(model_op)

s3://beercafe-ml-bucket/sagemaker-course/iris/model


In [16]:
model_img = sagemaker.image_uris.retrieve("xgboost", region, "latest")
print(model_img)

811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest


In [17]:
base_job_name = "iris-xgboost-"

In [19]:
xgb_model = sagemaker.estimator.Estimator(
    image_uri=model_img,
    role=role,
    base_job_name=base_job_name,
    instance_count=1,
    instance_type="ml.m4.xlarge",
    output_path=model_op,
    sagemaker_session=sagemaker.Session(),
    volume_size=5
)

In [20]:
xgb_model.set_hyperparameters(
    num_class=3, max_depth=5, num_round=10, objective="multi:softmax",
)

In [21]:
job_name = base_job_name + datetime.today().strftime("%Y-%m-%d-%H-%M-%S")
print(job_name)

iris-xgboost-2024-04-16-22-00-10


In [22]:
xgb_model.fit({"train": train_ip, "validation": valid_ip}, wait=True, job_name=job_name)

INFO:sagemaker:Creating training-job with name: iris-xgboost-2024-04-16-22-00-10


2024-04-16 16:30:47 Starting - Starting the training job...
2024-04-16 16:31:12 Starting - Preparing the instances for training......
2024-04-16 16:32:13 Downloading - Downloading input data...
2024-04-16 16:32:43 Downloading - Downloading the training image...
2024-04-16 16:33:33 Training - Training image download completed. Training in progress...Arguments: train
[2024-04-16:16:33:44:INFO] Running standalone xgboost training.
[2024-04-16:16:33:44:INFO] File size need to be processed in the node: 0.0mb. Available memory size in the node: 8491.82mb
[2024-04-16:16:33:44:INFO] Determined delimiter of CSV input is ','
[16:33:44] S3DistributionType set as FullyReplicated
[16:33:44] 100x4 matrix with 400 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2024-04-16:16:33:44:INFO] Determined delimiter of CSV input is ','
[16:33:44] S3DistributionType set as FullyReplicated
[16:33:44] 50x4 matrix with 200 entries loaded from /opt/ml/input/data/validation?forma

In [23]:
!aws s3 ls {bucket}/{prefix}/model/{job_name}/

                           PRE debug-output/
                           PRE output/
                           PRE profiler-output/
